In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [2]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utilities import aggregate_feature_calculators
from utilities import aggregate_feature_calculators_setting as aggcal
from utilities.parallel import Parallel

In [6]:
import sys
!{sys.executable} -m pip install openbabel

  Using cached https://files.pythonhosted.org/packages/9c/30/eb9c3d3d3b86981f6c6a7b8eceb6f4a13b9a12673efbc842b7cebe0ce39a/openbabel-2.4.1.tar.gz
  ERROR: Complete output from command /usr/bin/python3 -u -c 'import setuptools, tokenize;__file__='"'"'/tmp/pip-install-osebj045/openbabel/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-tjox3k9i --python-tag cp35:
  ERROR: running bdist_wheel
  running build
  running build_ext
  Guessing Open Babel location:
  - include_dirs: ['/usr/include/python3.5m', '/usr/local/include/openbabel-2.0']
  - library_dirs: ['/usr/local/lib']
  building '_openbabel' extension
  swigging openbabel-python.i to openbabel-python_wrap.cpp
  swig -python -c++ -small -O -templatereduce -naturalvar -I/usr/include/python3.5m -I/usr/local/include/openbabel-2.0 -o openbabel-python_wrap.cpp openbabel-python.i
  open

ERROR: Command "/usr/bin/python3 -u -c 'import setuptools, tokenize;__file__='"'"'/tmp/pip-install-osebj045/openbabel/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-9g_h__z0/install-record.txt --single-version-externally-managed --compile" failed with error code 1 in /tmp/pip-install-osebj045/openbabel/
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import os
import pandas as pd
import numpy as np
import math

from tqdm import tqdm_notebook, tqdm

import openbabel

ImportError: No module named 'tqdm'

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
file_folder =  '../../data/input'
os.listdir(file_folder)

['sample_submission.csv',
 'magnetic_shielding_tensors.csv',
 'potential_energy.csv',
 'scalar_coupling_contributions.csv',
 'dipole_moments.csv',
 'mulliken_charges.csv',
 'train.csv',
 'test.csv',
 'structures.csv',
 'structures']

In [17]:
train = pd.read_csv(f'{file_folder}/train.csv')
test = pd.read_csv(f'{file_folder}/test.csv')
magnetic_shielding_tensors = pd.read_csv(f'{file_folder}/magnetic_shielding_tensors.csv')
dipole_moments = pd.read_csv(f'{file_folder}/dipole_moments.csv')
mulliken_charges = pd.read_csv(f'{file_folder}/mulliken_charges.csv')
potential_energy = pd.read_csv(f'{file_folder}/potential_energy.csv')
scalar_coupling_contributions = pd.read_csv(f'{file_folder}/scalar_coupling_contributions.csv')
structures = pd.read_csv(f'{file_folder}/structures.csv')

In [18]:
x = structures.groupby('molecule_name').atom_index.max().reset_index(drop=False)
x.columns = ['molecule_name','totalatoms']
x.totalatoms+=1
train = train.merge(x,on='molecule_name')
# train = train[train.molecule_name=='dsgdb9nsd_000001']

In [ ]:
test = test.merge(x,on='molecule_name')

In [7]:
obConversion = openbabel.OBConversion()
obConversion.SetInFormat("xyz")
structdir=f'{file_folder}/structures/'
mols=[]
mols_files=os.listdir(structdir)
mols_index=dict(map(reversed,enumerate(mols_files)))
for f in mols_index.keys():
    mol = openbabel.OBMol()
    obConversion.ReadFile(mol, structdir+f) 
    mols.append(mol)

In [66]:
def _worker(item):
    
    m = item[0]
    groupdf = item[1]
    mol=mols[mols_index[m+'.xyz']]
    list_ = []
    for i in groupdf.index.values:
        
        id_ = int(groupdf.loc[i].id)
        totalatoms = groupdf.loc[i].totalatoms
        firstatomid = int(groupdf.loc[i].atom_index_0)
        secondatomid = int(groupdf.loc[i].atom_index_1)
        entrystats = {}
        entrystats['id'] = id_
        entrystats['totalatoms'] = totalatoms
        entrystats['scalar_coupling_constant'] = float(groupdf.loc[i].scalar_coupling_constant)
        entrystats['type'] = groupdf.loc[i]['type']
        a = mol.GetAtomById(firstatomid)
        b = mol.GetAtomById(secondatomid)
        entrystats['molecule_name'] = m
        entrystats['atom_index_0'] = firstatomid
        entrystats['atom_index_1'] = secondatomid
        entrystats['bond_distance'] = a.GetDistance(b)
        entrystats['bond_atom'] = b.GetType()

        #Put the tertiary data in order of distance from first hydrogen
        tertiarystats = {}
        for j,c in enumerate(list(set(range(totalatoms)).difference(set([firstatomid,secondatomid])))):
            tertiaryatom = mol.GetAtomById(c)
            tp = tertiaryatom.GetType()
            dist = a.GetDistance(tertiaryatom)
            ang = a.GetAngle(b,tertiaryatom)*math.pi/180
            while(dist in tertiarystats):
                dist += 1e-15
                # print('Duplicates!',m,j,dist)
            tertiarystats[dist] = [tp,dist,ang]
        
        for k, c in enumerate(sorted(tertiarystats.keys())):
            entrystats['tertiary_atom_'+str(k)] = tertiarystats[c][0]
            entrystats['tertiary_distance_'+str(k)] = tertiarystats[c][1]
            entrystats['tertiary_angle_'+str(k)] = tertiarystats[c][2]
        list_.append(entrystats)
    return list_


def extract_features(df):
    
    molecule_names = df.molecule_name.unique().tolist()
    N_ = len(molecule_names)
    segment = N_//100 if N_ > 100 else 1
    df_ = pd.DataFrame()
    for start in tqdm_notebook(range(0, N_, segment)):
        end = start+segment if start+segment < N_ else N_
        res = Parallel(_worker,{}).run(df[df['molecule_name'].isin(molecule_names[start:end])].groupby('molecule_name'), n_jobs=16)
        df_i = pd.DataFrame(res)
        df_ = pd.concat([df_, df_i], axis=0)
    df_ = df_.sort_values(by=['id'])
    return df_


In [67]:
df_train = extract_features(train)

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/ipykernel/__main__.py:56: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [69]:
print(df_train.shape, train.shape)
df_train.head()

(85003, 90) (4658147, 7)


,atom_index_0,atom_index_1,bond_atom,bond_distance,id,molecule_name,scalar_coupling_constant,tertiary_angle_0,tertiary_angle_1,tertiary_angle_10,...,tertiary_distance_26,tertiary_distance_3,tertiary_distance_4,tertiary_distance_5,tertiary_distance_6,tertiary_distance_7,tertiary_distance_8,tertiary_distance_9,totalatoms,type
0,4,0,C3,1.091948,9,dsgdb9nsd_000001,84.80950,1.910643,1.910637,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,1JHC
1,3,0,N3,1.017208,15,dsgdb9nsd_000002,32.69050,1.840223,1.840227,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1JHN
2,1,2,HO,1.513358,16,dsgdb9nsd_000003,-9.94641,0.665706,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2JHH
3,2,1,N1,2.218346,18,dsgdb9nsd_000005,5.18246,0.000447,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2JHN
14,7,1,C3,1.094968,45,dsgdb9nsd_000007,83.54860,1.874989,1.875022,NaN,...,NaN,2.543366,2.543368,3.09598,NaN,NaN,NaN,NaN,8,1JHC


In [ ]:
df_test = extract_features(test)

In [12]:
train = reduce_mem_usage(train)

Mem. usage decreased to 408.70 Mb (54.0% reduction)


In [15]:
test = reduce_mem_usage(test)

Mem. usage decreased to 215.05 Mb (53.1% reduction)


In [18]:
train.drop(columns=['molecule_name','atom_index_0','atom_index_1','type','scalar_coupling_constant']).to_pickle('../../data/feats/molecular-properties-eda-and-models_train.pkl')

In [19]:
test.drop(columns=['molecule_name','atom_index_0','atom_index_1','type']).to_pickle('../../data/feats/molecular-properties-eda-and-models_test.pkl')